In [37]:
%pylab inline
from snakebite.client import Client
import pyaml as yaml
import sys
import pythonrun
reload(pythonrun)
import os
import seaborn as sns
from sklearn import metrics
import scala_python_endive_wrapper
reload(scala_python_endive_wrapper)
from scala_python_endive_wrapper import *
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
hdfsclient = Client("amp-spark-master.amp", 8020, use_trash=False)
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')
logpath = "/tmp/log"

In [5]:
executor_mem = "150g"
cores_per_executor = 32
num_executors = 14
EGR1_PATH = '/user/vaishaal/endive-data/EGR1'

In [46]:
# this is where the magic happens
results = [] 
res = run_kitchensink_featurize_pipeline(EGR1_PATH,
                           "/tmp/filters.csv", 
                           logpath, 
                           seed=0,
                           filter_gen_gen=make_gaussian_filter_gen,
                           cores_per_executor=cores_per_executor,
                           sample=1.0,
                           executor_mem=executor_mem,
                           num_partitions=cores_per_executor*num_executors,
                           featuresOutput="/user/vaishaal/featurizedWindows/simple/tiny/replicate-1",
                           num_executors=num_executors)
results.append(res)

(256, 32)


In [8]:
chromosomes = map(lambda x: "chr{0}".format(x+1), range(23))
chromosomes.append('chrX')
chromosomes.append('chrY')
# CHANGE ME BASED ON ENUM + TF
egr1_cell_types = string_to_enum_celltypes(['GM12878', 'H1hESC', 'HCT116', 'MCF7'])

In [47]:
results_df = cross_validate("/user/vaishaal/featurizedWindows/simple/tiny/replicate-1", 
                            hdfsclient, 
                            chromosomes, 
                            egr1_cell_types,
                            numHoldOutChr=1, 
                            numHoldOutCell=1,
                            num_folds=1, 
                            negativeSamplingFreqs=[0.01],
                            regs=[1e-8, 1e-4, 1e-2])
    

FOLD 0
HOLDING OUT CHROMOSOMES ['chr6']
HOLDING OUT CELL TYPES [3]
RUNING SOLVER WITH REG=1e-08


Exception: invocation terminated with non-zero exit status

In [43]:
results_df

,negativeSamplingFreq,reg,test_celltypes,test_chromosomes,train_auPRC,train_auROC,train_recall_at_05_fdr,train_recall_at_10_fdr,train_recall_at_25_fdr,train_recall_at_50_fdr,val_auPRC,val_auROC,val_recall_at_05_fdr,val_recall_at_10_fdr,val_recall_at_25_fdr,val_recall_at_50_fdr
0,0.01,1.000000e-08,[3],[chr6],0.775019,0.905470,0.262909,0.499273,0.663636,0.824364,0.627768,0.819112,0.379310,0.379310,0.482759,0.586207
1,0.01,1.000000e-04,[3],[chr6],0.775245,0.907819,0.115273,0.508727,0.683636,0.828000,0.573658,0.799245,0.206897,0.206897,0.482759,0.551724
2,0.01,1.000000e-02,[3],[chr6],0.783608,0.907657,0.375273,0.518909,0.682182,0.825818,0.631226,0.818207,0.275862,0.275862,0.482759,0.689655
3,0.10,1.000000e-08,[3],[chr6],0.411634,0.880707,0.000000,0.000000,0.095636,0.482545,0.252295,0.811122,0.103448,0.103448,0.103448,0.137931
4,0.10,1.000000e-04,[3],[chr6],0.414045,0.879894,0.000000,0.000000,0.099273,0.485818,0.196092,0.812070,0.000000,0.000000,0.000000,0.103448
5,0.10,1.000000e-02,[3],[chr6],0.418043,0.880065,0.000000,0.000000,0.109455,0.486182,0.202571,0.813127,0.000000,0.000000,0.103448,0.137931
6,1.00,1.000000e-08,[3],[chr6],0.016197,0.851609,0.000000,0.000000,0.000000,0.000000,0.006556,0.791753,0.000000,0.000000,0.000000,0.000000
7,1.00,1.000000e-04,[3],[chr6],0.016197,0.851609,0.000000,0.000000,0.000000,0.000000,0.006556,0.791753,0.000000,0.000000,0.000000,0.000000
8,1.00,1.000000e-02,[3],[chr6],0.016197,0.851609,0.000000,0.000000,0.000000,0.000000,0.006556,0.791753,0.000000,0.000000,0.000000,0.000000
